In [ ]:
import tensorboard
import numpy as np
import torch

## Teacher:

# !CUDA_VISIBLE_DEVICES="" tensorboard --logdir=./cardiac_DCTeacherUNet/models_local/summary --port 4008 --host 172.16.101.10
# !CUDA_VISIBLE_DEVICES="" tensorboard --logdir=./exp/Seg_UNet_base/summary --port 4008 --host 172.16.101.53
!CUDA_VISIBLE_DEVICES="" tensorboard --logdir=/media/hticimg/data1/Data/MRI/datasets/calgary_dataset/experiments/calgary/sr3 --port 4007 --host 172.16.101.61

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

TensorBoard 2.9.1 at http://172.16.101.61:4007/ (Press CTRL+C to quit)
[2022-12-08T05:24:06Z WARN  rustboard_core::run] Read error in /media/hticimg/data1/Data/MRI/datasets/calgary_dataset/experiments/calgary/sr3/Resol_kd_cc/summary/events.out.tfevents.1670387897.hticimg: ReadRecordError(BadLengthCrc(ChecksumError { got: MaskedCrc(0x07980329), want: MaskedCrc(0x00000000) }))


In [ ]:
import numpy as np
# import basics.slice_view as slice_view
import matplotlib.pyplot as plt


import h5py
# filename = "/media/hticimg/data1/Data/MRI/datasets/calgary_dataset/cartesian/train/acc_4x/e13991s3_P01536.7.h5"
filename = "/media/hticimg/data1/Data/MRI/datasets/calgary_dataset/train/acc_4x/e13991s3_P01536.7.h5"


with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[2]
    print(list(f.keys())[1])
    print(f['img_volus_4x'].shape,f['kspace_volus_4x'].shape,f['volfs'].shape)

    # Get the data
#     data = list(f[a_group_key])
#     data_T2 = np.asarray(f[a_group_key])
    coord = np.asarray(f['img_volus_4x'])
    img = np.asarray(f['kspace_volus_4x'])
    img_main = np.asarray(f['volfs'])


In [ ]:
img1=img_main[50,:,:,0]
# print(np.unique(img1))
print(img1.shape)
# plt.imshow(img1,'gray')

In [ ]:
mask = np.zeros([256,256])
mask[128-32:128+32,128-32:128+32] = 1
# mask = np.zeros([2,256,256])
# mask[:,128-32:128+32,128-32:128+32] = 1

In [ ]:
kspace = np.fft.fft2(img1,norm='ortho')
kspace_shifted = np.fft.fftshift(kspace)
truncated_kspace = mask * kspace_shifted
lr_img = np.abs(np.fft.ifft2(truncated_kspace,norm='ortho'))
# print(truncated_kspace.shape,mask.shape,kspace_shifted.shape)
plt.imshow(lr_img,'gray')

In [ ]:
mask = np.ones([256,256])
mask[128-32:128+32,128-32:128+32] = 0
kspace = np.fft.fft2(img1,norm='ortho')
kspace_shifted = np.fft.fftshift(kspace)
truncated_kspace = mask * kspace_shifted
lr_img = np.abs(np.fft.ifft2(truncated_kspace,norm='ortho'))
# print(truncated_kspace.shape,mask.shape,kspace_shifted.shape)
plt.imshow(lr_img,'gray')

In [ ]:
import numpy as np

img_array = np.load('/media/hticimg/data1/Data/MRI/usmasks/calgary_dataset/cartesian/mask_4x.npy')
from matplotlib import pyplot as plt

plt.imshow(img_array, cmap='gray')
plt.show()

In [ ]:
# # print(coord,img.shape,mask.shape)

# plt.imshow(img,'gray')
# # plt.imshow(mask,'gray')
# plt.show()

# plt.imshow(mask,'gray')
# plt.show()
# # import cv2

In [ ]:
import SimpleITK as sitk
import sys
import glob
import os
# %matplotlib inline
import matplotlib.pyplot as plt

# sys.path.insert(0,'./basics/')
# import basics.slice_view as slice_view

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F

# torchsummary and torchvision
from torchsummary import summary
from torchvision.utils import save_image

# matplotlib stuff
import matplotlib.pyplot as plt
import matplotlib.image as img

# numpy and pandas
import numpy as np
import pandas as pd

# Common python packages
import datetime
import os
import sys
import time

import pathlib
import random

import h5py
# from torch.utils.data import Dataset
import torch
from skimage import feature
# from utils import npComplexToTorch

In [ ]:
from models import *

In [ ]:
x = torch.rand((1,2,256,256))
teacher = TeacherVDSR()
y=teacher(x)
print(y[0].shape,y[1].shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
import plotter
from pathlib import Path
import os


class OTSolver(object):
    def __init__(self, dist1, dist2, marginal1=None, marginal2=None, ground_cost='l2', logdir='results'):
        self.dist1 = dist1
        self.dist2 = dist2
        nsamples1 = dist1.shape[0]
        nsamples2 = dist2.shape[0]
        self.nsamples1 = nsamples1
        self.nsamples2 = nsamples2

        if marginal1 is None:
            self.marginal1 = np.array([1/nsamples1 for i in range(nsamples1)])
        else:
            self.marginal1 = marginal1

        if marginal2 is None:
            self.marginal2 = np.array([1/nsamples2 for i in range(nsamples2)])
        else:
            self.marginal2 = marginal2
        self.marginal1 = np.expand_dims(self.marginal1, axis=1)
        self.marginal2 = np.expand_dims(self.marginal2, axis=1)

        self.ground_cost = ground_cost
        assert ground_cost in ['l2']
        self.logdir = logdir
        Path(self.logdir).mkdir(parents=True, exist_ok=True)

    def form_cost_matrix(self, x, y):
        if self.ground_cost == 'l2':
            return np.sum(x ** 2, 1)[:, None] + np.sum(y ** 2, 1)[None, :] - 2 * x.dot(y.transpose())

    def solve(self, plot=False):
        C = self.form_cost_matrix(self.dist1, self.dist2)
        P = cp.Variable((self.nsamples1, self.nsamples2))

        u = np.ones((self.nsamples2, 1))
        v = np.ones((self.nsamples1, 1))
        constraints = [0 <= P, cp.matmul(P, u) == self.marginal1, cp.matmul(P.T, v) == self.marginal2]

        objective = cp.Minimize(cp.sum(cp.multiply(P, C)))
        prob = cp.Problem(objective, constraints)
        result = prob.solve()
        coupling = P.value

        print("Number of non-zero values in P: {} (n + m-1 = %d)".format(len(coupling[coupling > 1e-5]),
                                                                         self.nsamples1 + self.nsamples2 - 1))
        print("Objective function: {}".format(objective.value))

        if plot:
            print('Generating plots ...')
            plotter.generate_scatter_plots(self.dist1, self.dist2,
                                           '{}/orig.png'.format(self.logdir))
            plotter.generate_scatter_plots_with_coupling(self.dist1, self.dist2, coupling,
                                                         '{}/coupling.png'.format(self.logdir))

        OT_cost = objective.value
        return OT_cost

In [ ]:
# !pip install cvxpy
# !pip install plotter
# !pip install tikzplotlib


In [ ]:
X = torch.ones((4,2,5,5))
Y = torch.zeros((4,2,5,5))
X=X.permute(0,1,2,)
# y=x
y=Y.numpy()
x=X.numpy()

x=x.reshape(,25)
y=y.reshape(8,25)

In [ ]:
OT_cost=OTSolver(x,y)

In [ ]:
c=OT_cost.solve()

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import time
from torch import nn

In [ ]:
class REMD(nn.Module):
    """Relaxed Eatth Mover Distance from the paper Model Compression using OT 
    Code from author: https://github.com/sunniesuhyoung/DST/blob/master/loss.py"""
    def __init__(self):
        super(REMD, self).__init__()
    
    def forward(self, f_s, f_t):
        return self.remd_loss(f_s, f_t)

    def pairwise_distances_cos(self,x, y):
        x_norm = torch.sqrt((x**2).sum(1).view(-1, 1))
        y_t = torch.transpose(y, 0, 1)
        y_norm = torch.sqrt((y**2).sum(1).view(1, -1))
        dist = 1.-torch.mm(x, y_t)/x_norm/y_norm
        return dist

    def pairwise_distances_sq_l2(self,x, y):
        x_norm = (x**2).sum(1).view(-1, 1)
        y_t = torch.transpose(y, 0, 1)
        y_norm = (y**2).sum(1).view(1, -1)
        dist = x_norm + y_norm - 2.0 * torch.mm(x, y_t)
        return torch.clamp(dist, 1e-5, 1e5)/x.size(1)

    def distmat(self,x, y):
        M = self.pairwise_distances_cos(x, y)
            # M = torch.sqrt(pairwise_distances_sq_l2(x, y))
        return M


    def remd_loss(self,X, Y):
        d = X.shape[1]
        e = X.shape[0]
        print(d,e,X.shape,Y.shape)
        X = X.transpose(0, 1).contiguous().view(d*e, -1).transpose(0, 1)
        Y = Y.transpose(0, 1).contiguous().view(d*e, -1).transpose(0, 1)
        print(X.shape)
        print(Y.shape)
        CX_M = self.distmat(X, Y)
        m1, m1_inds = CX_M.min(1)
        m2, m2_inds = CX_M.min(0)

        loss = torch.max(m1.mean(), m2.mean())
        return loss

In [ ]:
loss = REMD()

In [ ]:
X = torch.randn((4,2,5,5))
Y = torch.randn((4,2,5,5))
# print(X,Y)

In [ ]:
# print(X.shape)

ot=loss(X,Y)
ot

In [ ]:
y=Y.numpy()
x=X.numpy()

x=x.reshape(8,25)
y=y.reshape(8,25)
OT_cost=OTSolver(x,y)
c=OT_cost.solve()

In [ ]:
x=x.reshape(25,8)
y=y.reshape(25,8)
OT_cost=OTSolver(x,y)
c=OT_cost.solve()